In [21]:
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.cm as cm 

data_file = os.path.join("new_txn.pkl")

In [22]:
import torch
from torch.utils.data import Dataset, DataLoader
# from keras.preprocessing.sequence import pad_sequences

dtype = torch.float32
device = torch.device('cuda:0')
device

device(type='cuda', index=0)

In [23]:
# data_file = os.path.join(DATA_DIR, 'prod.pkl')
data = pd.read_pickle(data_file)
data

,訂單編號,下單日期,時間戳記,psid,asid,user_id,channel_name,商品id,商品名稱,樣式,...,付款單號,專屬折扣,運費,收款金額,付款方式,運送方式,寄送地址,場次,date,item_name
0,BD220301-86612477,2022-03-01,00:04:56,2144742712282981,2912039312151026,10891,邦成-自倉（C倉出貨付款全開）,6492974,（免運）(闆)公主 0228 A01-(限+1)(免運)(3件)100%石墨烯遠紅外線棉被(...,NaN,...,PA220301-15817276,0,0,14044,LINE Pay,一般宅配,桃園市蘆竹區大興十四街77號,20220228(公主x深-重播0221 888C倉晚上A1)-原,2022-03-01,公主 0228 A01-(限+1)(免運)(3件)100%石墨烯遠紅外線棉被(180X210...
1,BD220301-86612495,2022-03-01,00:05:00,4880729621967911,1681086192072457,10891,邦成-自倉（C倉出貨付款全開）,6493322,大師兄 0228 A01-金屬玻璃調味瓶*1,NaN,...,PA220301-15831018,0,150,2048,貨到付款,一般宅配,新北市樹林區俊英街155巷43弄15號4樓之3,20220228(大師兄xC倉深夜場1)-儒/川/駿,2022-03-01,大師兄 0228 A01-金屬玻璃調味瓶*1
2,BD220301-86612507,2022-03-01,00:05:00,3861530257224160,2836025813335298,10891,邦成-自倉（C倉出貨付款全開）,6493322,大師兄 0228 A01-金屬玻璃調味瓶*1,NaN,...,PA220301-15815246,0,0,595,自取,一般宅配,自取點-B倉,20220228(大師兄xC倉深夜場1)-儒/川/駿,2022-03-01,大師兄 0228 A01-金屬玻璃調味瓶*1
3,BD220301-86612510,2022-03-01,00:05:00,3521679651227760,3120032084685049,10891,邦成-自倉（C倉出貨付款全開）,6493322,大師兄 0228 A01-金屬玻璃調味瓶*1,NaN,...,PA220303-15883127,0,0,14664,匯款,一般宅配,南投縣竹山鎮大同街111號,20220228(大師兄xC倉深夜場1)-儒/川/駿,2022-03-03,大師兄 0228 A01-金屬玻璃調味瓶*1
4,BD220301-86612516,2022-03-01,00:05:02,2542831359072575,3383335041684491,10891,邦成-自倉（C倉出貨付款全開）,6478836,888 0227 A18-(5盒)BRE 療肺草暢呼吸飲(8包/盒)*5,NaN,...,PA220303-15877459,0,0,3866,自取,一般宅配,自取點-B倉,20220227(888xC倉下午場1)-玲/哲/砡,2022-03-03,888 0227 A18-(5盒)BRE 療肺草暢呼吸飲(8包/盒)*5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5788747,BD220430-96897943,2022-04-30,14:57:03,NaN,2246133855485630,4793,279-🛒現貨直播車,6756827,【商城】Goongbe宮中秘策48小時 寶寶保濕霜30ml(202203170001),NaN,...,PA220430-17836184,0,0,215,自取,一般宅配,自取點-總店,NaN,2022-04-30,Goongbe宮中秘策48小時 寶寶保濕霜30ml(202203170001)
5788748,BD220430-96898373,2022-04-30,15:01:56,NaN,2246133855485630,4793,279-🛒現貨直播車,7277971,【商城】Dripoドリポ牧場 **印度** 紅茶牛乳(1條),NaN,...,PA220430-17836184,0,0,215,自取,一般宅配,自取點-總店,NaN,2022-04-30,Dripoドリポ牧場 **印度** 紅茶牛乳(1條)
5788751,BD220430-96908225,2022-04-30,16:13:23,NaN,2717578198260964,4793,279-🛒現貨直播車,6556105,【商城】亞大T8銀耳 冰糖銀耳羹(一瓶)(202112270007),NaN,...,PA220501-17866137,0,60,2520,貨到付款,一般宅配,桃園市楊梅區中山北路1段360號,NaN,2022-05-01,亞大T8銀耳 冰糖銀耳羹(一瓶)(202112270007)
5788752,BD220430-96910480,2022-04-30,16:32:37,NaN,2896144450612065,4793,279-🛒現貨直播車,7145579,【商城】Binaca舌苔刷(一支)(202204110007),NaN,...,PA220430-17838652,0,70,715,超取付,7-11C2C,7-ELEVEN 遠德門市（地址：台南市新化區北勢里中正路1211號 / 電話：--）,NaN,2022-04-30,Binaca舌苔刷(一支)(202204110007)


In [24]:
item_names = list(data['item_name'].unique())
item_names

['公主 0228 A01-(限+1)(免運)(3件)100%石墨烯遠紅外線棉被(180X210)*3-價錢後台看',
 '大師兄 0228 A01-金屬玻璃調味瓶*1',
 '888 0227 A18-(5盒)BRE 療肺草暢呼吸飲(8包/盒)*5',
 '公主 0228 A01-(限+3)(免運/2件組)諾貝爾物理獎-舒眠博士石墨烯毯-雙人(150*190cm)*1+龜鹿五臻寶(30g/罐)-(10入裝)*1',
 '大師兄 0228 A06-秋紅饗胡椒豬肚雞 (450g)*1',
 '888 0227 A17-(5罐)BRE台農膠原蛋白肽粉(150g/罐)*5-結單缺貨刪單不通知',
 "888 0225 A10-(6盒)cab's人蔘黑棗雞精(8包/盒)*6-價錢後台看",
 '888 0228 A03-洗衣機槽專用強力清潔錠(16入/盒)*1',
 '888 0227 A04-FOUR SEASONS擴香瓶200ML(隨機)*1-結單缺貨刪單不通知',
 '公主 0228 A09-(8盒)BRE阿育吠陀 七脈輪淨化植萃膠囊(30顆/盒)*8',
 '大師兄 0228 A02-25*25 加大 珊瑚絨抹布(10入)*1',
 '畢哥 0228 A05-(6件組)PRINCESS PARTY 超皙錠(30粒/盒)*5+Mixzuzu 美容膠原蛋白莓妍飲190ml(24瓶/箱)－草莓奶昔*1--結單缺貨刪單不通知',
 '888 0228 A02-(9包)日本P&G Ariel 3.3倍炭酸4D洗衣膠球(39顆/袋)-味道隨機*9',
 '888 0228 A01-純水濕紙巾 (80抽)(隨機)-後台看幾包',
 '大師兄 0228 A03-繽紛平煎鍋26CM(顏色款式隨機)*1',
 '公主 0228 A10-和秋 海苔芝麻雞肉鬆(200g)(效期2022/08)*1',
 '公主 0228 A07-(10雙)ASEDO 台灣亞斯多石墨烯抗臭除菌科技短襪(23~28CM)(一雙)(黑)*10',
 '大師兄 0228 A04-和秋 海苔芝麻雞肉鬆(200g)*1',
 '大師兄 0228 A05-Snoopy潮玩藝術-丹寧圍裙(長100CM*寬60CM)*1',
 '公主 0228 A11-(5盒)BRE 療肺草暢呼吸飲(8包/盒)*5',
 '公主 0226

In [25]:
item_name_max_len = max([len(i) for i in item_names])
item_name_max_len

101

In [26]:
from transformers import (
  BertTokenizerFast,
  AutoModel,
)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')

# model = AutoModel.from_pretrained('ckiplab/bert-base-chinese-ws')
model = AutoModel.from_pretrained('ckiplab/bert-base-chinese')
model.to(device)

Some weights of the model checkpoint at ckiplab/bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ckiplab/bert-base-chinese and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dens

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

### 使用BERT取得商品名稱的向量

In [27]:
pad_token_item_names = tokenizer.batch_encode_plus(item_names, padding=True)['input_ids']
pad_token_item_names

[[101,
  1062,
  712,
  12522,
  8156,
  100,
  118,
  113,
  7361,
  116,
  122,
  114,
  113,
  1048,
  6880,
  114,
  113,
  124,
  816,
  114,
  8135,
  110,
  4767,
  1874,
  4179,
  6895,
  5148,
  1912,
  5221,
  3469,
  6158,
  113,
  100,
  114,
  115,
  124,
  118,
  1019,
  7092,
  2527,
  1378,
  4692,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [101,
  1920,
  2374,
  1040,
  12522,
  8156,
  100,
  118,
  7032,
  2253,
  4390,
  4461,
  6310,
  1456,
  4486,
  115,
  122,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [28]:
pad_token_item_names = torch.tensor(pad_token_item_names, device=device)
pad_token_item_names

tensor([[  101,  1062,   712,  ...,     0,     0,     0],
        [  101,  1920,  2374,  ...,     0,     0,     0],
        [  101,  9389, 12522,  ...,     0,     0,     0],
        ...,
        [  101,  1259,  6181,  ...,     0,     0,     0],
        [  101,   100,  4288,  ...,     0,     0,     0],
        [  101,   765,  1920,  ...,     0,     0,     0]], device='cuda:0')

In [29]:
class BertTokenDataset(Dataset):
    def __init__(self, tokens):
        self.tokens = tokens
        
    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        return self.tokens[idx]

token_dataset = BertTokenDataset(pad_token_item_names)
 # 可以根據顯存大小改這裡的 batch size，讓速度快一點（跑不動的話就只能調小一點...）
token_dataloader = DataLoader(token_dataset, shuffle=False, batch_size=32)

In [30]:
%%time
item_name_embedding = []
for token in tqdm(token_dataloader):
    embedding = model(token).pooler_output
    embedding = embedding.tolist()
    item_name_embedding.extend(embedding)
item_name_embedding = np.array(item_name_embedding)

  0%|          | 0/5361 [00:00<?, ?it/s]

CPU times: user 11min 22s, sys: 1.84 s, total: 11min 24s
Wall time: 11min 21s


In [31]:
item_name_embedding.shape

(171546, 768)

In [38]:
iname2embedding = {n:emb for n, emb in zip(item_names, item_name_embedding)}

In [42]:
np.save('jambo_iname2embedding.npy', iname2embedding)

In [43]:
test = np.load('jambo_iname2embedding.npy', allow_pickle=True)[None][0]

In [44]:
test['澳洲Eureka天然10%水溶性尤加利精油500ml']

array([ 4.52520519e-01,  4.48803574e-01, -1.17524071e-02, -3.85705858e-01,
       -7.74326026e-02, -5.56467831e-01,  6.52689338e-01,  3.91783342e-02,
       -4.09322120e-02, -3.51034589e-02,  1.34102643e-01, -6.29035890e-01,
       -3.29120517e-01,  1.20923690e-01,  1.88460678e-01, -2.09421962e-02,
        6.87014639e-01, -5.15987724e-02,  4.31619853e-01, -2.61927038e-01,
        4.48834658e-01, -2.04338491e-01, -1.93931013e-01,  1.26866296e-01,
       -6.10986173e-01,  2.54999489e-01,  3.52517098e-01,  1.16788864e-01,
       -5.39703369e-01,  6.67024478e-02, -1.34121954e-01,  1.15128130e-01,
       -4.36380446e-01,  3.56863216e-02,  7.41245821e-02,  2.61862844e-01,
       -2.70205706e-01, -2.58509994e-01, -3.67636085e-01, -4.33911115e-01,
       -1.79799452e-01,  5.99434495e-01,  2.64127344e-01, -3.54311645e-01,
        4.72281307e-01,  1.50280997e-01, -3.43477577e-01,  2.45001361e-01,
       -3.10740203e-01,  4.04418141e-01,  3.53185326e-01,  2.22031862e-01,
        2.29290500e-01, -

In [ ]:
len()

In [20]:
list(iname2embedding.keys())[100:120]

['畢哥 0301 A08-Snoopy洗衣袋兩件組*1',
 'HC-和秋黃金白菜滷450g',
 '公主 0225 A04-(18件組)CHOCO印加果油複方軟膠囊(30顆/盒)*10+升級版 BRE魔塑果膠XS(8包/盒)*6+雙層玻璃杯(款式隨機)*2',
 '秀炮 0301 A01-(後台看數量)【樂一通】純水濕紙巾 (80抽)',
 '秀炮 0301 A02-(5罐)潔廁魔瓶220G(味道隨機)*5',
 '秀炮 0301 A03-(3組)北歐不鏽鋼防滑衣架 (10入)*3',
 '（免運）秀炮 0301 A04-(免運)(7件組)公主派對。雙龍鴕鳥精膠囊 (30顆/盒)*5+龜鹿五臻寶(30g/罐)-(10入裝)*2-價錢後台看',
 "秀炮 0301 A05-KA'FEN 精油洗手露 (500ml)(隨機)*1-結單缺貨刪單不通知",
 '公主 0211 A02-公主派對。青梅果*1',
 '公主 0220 A09-(10盒)健康曙光金盞花葉黃素PA加強配方膠囊(30粒/盒)(效期2024/02)*10',
 '（免運）(闆)888 0301 A01-(限+3)(免運/4件)諾貝爾物理獎-舒眠博士石墨烯毯-雙人(150*190cm)*1+龜鹿五臻寶(30g/罐)-(10入裝)*1-(結單禮:牛樟芝子實體原片*2)',
 '秀炮 0301 A08-(5條)CANDY-BC冰冰防蚊乳 (50ml)*5',
 '（免運）(闆)888 0301 A02-(免運)(石墨烯5件組)-價錢後台看-(結單禮:BRE 牛樟芝 子實體原片(1.3g±10%/朵) (1朵/盒)*2)',
 '(闆)公主 0129 A09-(6盒)公主派對。超時空美姬飲(10包/盒)*6',
 '（免運）(闆)888 0301 A03-(免運/預購/石墨烯8件組-價錢後台看)-(結單禮:BRE 牛樟芝 子實體原片(1.3g±10%/朵) (1朵/盒)*2)',
 '公主 0225 A16-和秋 海苔芝麻雞肉鬆(200g)*1',
 '秀炮 0301 A10-平口型點斷式垃圾袋(45X50CM)OPP袋裝(20抽*5捲/袋)(顏色隨機)*1',
 '秀炮 0301 A09-非動力式新黑科技石墨烯床墊<單人>(100*150CM)*1',
 '888 0301 A04-史努比史於經典-LED抽

In [21]:
iname2embedding['【商城】澳洲Eureka天然10%水溶性尤加利精油500ml']

array([ 5.47077179e-01, -1.03998452e-01,  7.17962027e-01, -4.76443827e-01,
        2.93173879e-01, -1.00719810e-01,  2.17953622e-01,  3.86685282e-01,
        5.94051361e-01,  5.69490671e-01,  3.77794474e-01, -1.14667773e-01,
        1.60344362e-01,  3.41246307e-01,  2.36063570e-01, -4.34189290e-01,
        4.88859229e-03, -1.23239681e-01,  2.21650496e-01, -3.14609915e-01,
       -1.69456720e-01,  2.84895748e-02,  1.64609104e-01,  5.99745274e-01,
        3.34352881e-01, -2.33788237e-01,  1.94201827e-01, -2.25733101e-01,
        6.33559763e-01,  5.83667338e-01,  8.44187811e-02, -1.02498583e-01,
       -1.89643502e-01, -1.33723423e-01,  3.72063458e-01, -4.59649526e-02,
        7.62003481e-01, -2.10526496e-01, -4.66295421e-01,  1.17787555e-01,
       -4.67444301e-01, -5.49636930e-02, -6.30799890e-01, -4.20374632e-01,
       -3.92928511e-01, -3.45702499e-01,  2.06197929e-02, -5.75189829e-01,
        1.42884538e-01,  7.39915192e-01, -2.82438211e-02, -9.65500399e-02,
        7.56164730e-01,  